In [1]:
import sys
sys.path.append("/workspaces/chisel/")

# Download NCBI dataset

In [2]:
import requests
import zipfile
import io
import os

In [3]:
# Step 1: Provide the URL to the zip file
url = "https://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/NCBI_corpus.zip"

# Step 2: Set the extraction destination
extract_to = "/workspaces/chisel/examples/data"
os.makedirs(extract_to, exist_ok=True)

# Step 3: Download and extract
response = requests.get(url)
if response.status_code == 200:
    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Extracted contents to: {extract_to}")
else:
    print(f"Failed to download file. Status code: {response.status_code}")

Extracted contents to: /workspaces/chisel/examples/data


In [3]:
annotations = []
with open("/workspaces/chisel/examples/data/NCBI_corpus_training.txt", "r") as f:
    for line in f.readlines():
        splits = line.split("\t")
        annotations.append({
            "id": splits[0].strip(),
            "text": " ".join(splits[1:]).strip().replace('<category="', '<category category="')
        })

# Pre-process with Chisel to Transformer format

In [8]:
# 📦 Imports
from transformers import AutoTokenizer
from chisel.extraction.parsers.html_tag_parser import HTMLTagParser
from chisel.extraction.tokenizers.hf_tokenizer import HFTokenizer
from chisel.extraction.chunkers.fixed_length_chunker import FixedLengthTokenChunker
from chisel.extraction.span_aligners.token_span_aligner import TokenSpanAligner
from chisel.extraction.labelers.bio_labeler import BIOLabeler
from chisel.extraction.labelers.label_encoder import SimpleLabelEncoder
from chisel.extraction.validators.validators import DefaultParseValidator, HFTokenAlignmentValidator
from chisel.extraction.exporters.dataset_exporter import DatasetExporter
from chisel.extraction.models.models import ChiselRecord
from chisel.extraction.models.models import EntitySpan
from typing import List

In [12]:
# 🔍 Components
parser = HTMLTagParser(label_strategy="attribute", attribute_name="category")
tokenizer = HFTokenizer(model_name="bert-base-cased")
aligner = TokenSpanAligner()
chunker = FixedLengthTokenChunker(max_tokens=512, overlap=0)
labeler = BIOLabeler()
label_encoder = SimpleLabelEncoder(label_to_id={
 'O': 0,
 'B-Modifier': 1,
 'I-Modifier': 2,
 'B-SpecificDisease': 3,
 'I-SpecificDisease': 4,
 'B-CompositeMention': 5,
 'I-CompositeMention': 6,
 'B-DiseaseClass': 7,
 'I-DiseaseClass': 8,
})

parse_validators = [DefaultParseValidator()]
label_validators = [HFTokenAlignmentValidator(tokenizer=tokenizer.tokenizer)]


exporter = DatasetExporter(output_path="data/ncbi_torch_dataset.pkl")

In [ ]:
processed_data = []

# 🔁 Pipeline loop
for example in annotations:
    text, entities = parser.parse(example["text"])
    
    for validator in parse_validators:
        validator.validate(text, entities)

    tokens = tokenizer.tokenize(text)
    token_entity_spans = aligner.align(entities, tokens)

    token_chunks, entity_chunks = chunker.chunk(tokens, token_entity_spans)

    for chunk_id, (toks, ents) in enumerate(zip(token_chunks, entity_chunks)):

        labels = labeler.label(toks, ents)
        encoded_labels = label_encoder.encode(labels)

        for validator in label_validators:
            validator.validate(toks, ents, labels)

        record = ChiselRecord(
                id=example["id"],
                chunk_id=chunk_id,
                text=tokenizer.tokenizer.decode([token.id for token in toks]),
                tokens=toks,
                input_ids=[token.id for token in toks],
                attention_mask=[1] * len(toks),
                entities=[tes.entity for tes in ents],
                bio_labels=labels,
                labels=encoded_labels
            )
        processed_data.append(record)

# export
exporter.export(processed_data)
print("✅ Export complete")

Token indices sequence length is longer than the specified maximum sequence length for this model (570 > 512). Running this sequence through the model will result in indexing errors


  Decoded actual: 'autosomal dominant'
  Decoded expected: 'autosomal dominant disorde'
  Decoded actual: 'absence of the seventh component of'
  Decoded expected: 'absence of the seventh component of complemen'


Saving the dataset (1/1 shards): 100%|██████████| 618/618 [00:00<00:00, 2962.95 examples/s]

✅ Export complete: data/ncbi_torch_dataset.pt
